In [1]:
!uv add youtube-transcript-api

Resolved 172 packages in 6.15s
Installed 1 package in 380ms
 + youtube-transcript-api==1.2.4


In [2]:
video_id = 'r0gHbW_MmaQ'

In [3]:
from youtube_transcript_api import YouTubeTranscriptApi

In [21]:
ytt_api = YouTubeTranscriptApi()
transcript = ytt_api.fetch(video_id)

In [8]:
transcript.snippets[-1]

FetchedTranscriptSnippet(text='you. Thanks. See you.', start=3526.079, duration=4.52)

In [10]:
import datetime

def format_timestamp(seconds: float) -> str:
    total_seconds = int(seconds)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, secs = divmod(remainder, 60)

    if hours > 0:
        return f"{hours}:{minutes:02}:{secs:02}"
    else:
        return f"{minutes:02}:{secs:02}"


In [11]:
format_timestamp(3526.079)

'58:46'

In [14]:
transcript_lines = []

for snippet in transcript.snippets:
    ts = format_timestamp(snippet.start)
    line = f'{ts} {snippet.text}'
    transcript_lines.append(line)

In [19]:
subtitles = '\n'.join(transcript_lines[:10])

In [20]:
print(subtitles[:400])

00:00 Hi everyone, welcome to the second
00:02 iteration of AI engineering boot camp.
00:05 So I renamed the course recently. It
00:08 used to be called AI boot camp but there
00:10 are so many boot camps and AI boot camps
00:12 these days. So I was thinking if there's
00:15 a way to somehow be different from these
00:17 courses and uh maybe you know me by uh
00:21 from courses u from free courses


In [22]:
def make_subtitles(transcript) -> str:
    lines = []

    for entry in transcript:
        ts = format_timestamp(entry.start)
        text = entry.text.replace('\n', ' ')
        lines.append(ts + ' ' + text)

    return '\n'.join(lines)

subtitles = make_subtitles(transcript)

In [32]:
from pathlib import Path

subtitles_file = Path(f'subtitles_{video_id}.txt')
subtitles_file.write_text(subtitles, encoding='utf-8')

56264

In [35]:
subtitles = subtitles_file.read_text(encoding='utf-8')

In [36]:
from openai import OpenAI
openai_client = OpenAI()

In [38]:
def llm(instructions, prompt, model='gpt-4o-mini'):
    messages = [
        {'role': 'system', 'content': instructions},
        {'role': 'user', 'content': prompt}
    ]

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text


In [69]:
instructions = """
Summarize the following youtube transcript and create a timestamped
list of video chaptes

The goal is to use this as the description under the YouTube video.
The output should be copy-pastable to youtube, so don't start the
summary with "the transcript provides...". 
Don't start with "Welcome...", "In this video," etc

It should contain 2-3 paragraphs of text.

Don't use any formatting in the response.

The correct name of the course is "AI Engineering Buildcamp" 

After the description, create the chapter list in the following format

<timestamp> <chapter name>

Make sure chapters are at least 3-4 minutes long but not longer than
6 minutes. The chapters should 
correspond to what's actually discussed in the video

The chapter should start where the concept is introduced, and should be very
precise - this is going to be used for youtube, and viewers will use it for
quick navigation around the video
"""

In [72]:
summary = llm(
    instructions=instructions,
    prompt=subtitles,
    model='gpt-4o-mini'
)

In [73]:
print(summary)

The AI Engineering Buildcamp focuses on a hands-on approach to building AI systems, emphasizing the engineering mindset. Participants are encouraged to engage in practical programming while adhering to best practices. The course structure entails a mix of core and optional materials, ensuring learners can navigate content according to their pace. The instructor introduces the curriculum, highlights the platform used, and addresses how weekly sessions will be organized, including office hours for additional support.

Over the coming weeks, the buildcamp will cover various topics, such as foundational concepts of large language models (LLMs), programming with APIs, implementing structured outputs, and building agents. Participants are to embark on mini projects and ultimately conceptualize their capstone projects, which will showcase their skills. Additionally, there's an emphasis on collaboration through a dedicated Slack community, where learners can share insights and ask questions th

<17:10> <Week 3: Agents and Interactive Learning>